In [24]:
import functools

import jax
import numpy as np
import scanpy as sc

from ot_pert.metrics import compute_mean_metrics, compute_metrics

In [3]:
adata = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [15]:
adata_train_orig = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [7]:
preds = adata.obsm["CPA_pred"]

In [41]:
def reconstruct_data(embedding: np.ndarray, projection_matrix: np.ndarray, mean_to_add: np.ndarray) -> np.ndarray:
    return np.matmul(embedding, projection_matrix.T) + mean_to_add


def project_data(data: np.ndarray, projection_matrix: np.ndarray, mean_to_subtract: np.ndarray) -> np.ndarray:
    return np.matmul(data - mean_to_subtract, projection_matrix)

In [42]:
OBSM_KEY_DATA_EMBEDDING = "X_pca"

In [37]:
adata_train = adata[adata.obs["split"] == "train"]
adata_test = adata[adata.obs["split"] == "test"]
adata_ood = adata[adata.obs["split"] == "ood"]

In [43]:
project_data_fn = functools.partial(
    project_data,
    projection_matrix=adata_train_orig.varm["PCs"],
    mean_to_subtract=adata_train_orig.varm["X_train_mean"].T,
)

In [44]:
adata_train_orig.varm["PCs"].shape

(2000, 50)

In [48]:
train_data_target_encoded = {}
train_data_target_decoded = {}
train_data_target_encoded_predicted = {}
train_data_target_decoded_predicted = {}

for cond in adata_train.obs["condition"].cat.categories:
    if cond == "control":
        continue
    train_data_target_encoded[cond] = adata_train[adata_train.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    train_data_target_decoded[cond] = adata_train[adata_train.obs["condition"] == cond].X.A
    pred_cpa = np.log1p(adata_train[adata_train.obs["condition"] == cond].obsm["CPA_pred"])
    train_data_target_decoded_predicted[cond] = pred_cpa
    train_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)


test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.A
    pred_cpa = np.log1p(adata_test[adata_test.obs["condition"] == cond].obsm["CPA_pred"])
    test_data_target_decoded_predicted[cond] = pred_cpa
    test_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)


ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm[OBSM_KEY_DATA_EMBEDDING]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.A
    pred_cpa = np.log1p(adata_ood[adata_ood.obs["condition"] == cond].obsm["CPA_pred"])
    ood_data_target_decoded_predicted[cond] = pred_cpa
    ood_data_target_encoded_predicted[cond] = project_data_fn(pred_cpa)

In [50]:
train_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, train_data_target_encoded, train_data_target_encoded_predicted
)
mean_train_metrics_encoded = compute_mean_metrics(train_metrics_encoded, prefix="encoded_train_")

train_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, train_data_target_decoded, train_data_target_decoded_predicted
)
mean_train_metrics_decoded = compute_mean_metrics(train_metrics_decoded, prefix="decoded_train_")

2024-04-16 18:19:54.641830: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [51]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [52]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [53]:
mean_train_metrics_encoded

{'encoded_train_r_squared': 0.09882947343231972,
 'encoded_train_sinkhorn_div_01': 27.670842895507814,
 'encoded_train_e_distance': 2.8356454493050745,
 'encoded_train_mmd': 12.157588424682617}

In [54]:
mean_train_metrics_decoded

{'decoded_train_r_squared': 0.7729072789319338,
 'decoded_train_sinkhorn_div_01': 83.83307525634766,
 'decoded_train_e_distance': 3.982077108849602,
 'decoded_train_mmd': 20.897135314941405}

In [55]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.22096380806799604,
 'encoded_test_sinkhorn_div_01': 21.72205924987793,
 'encoded_test_e_distance': 2.495100569970721,
 'encoded_test_mmd': 10.17115592956543}

In [56]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7646439309457859,
 'decoded_test_sinkhorn_div_01': 87.17083251953125,
 'decoded_test_e_distance': 4.067989317550506,
 'decoded_test_mmd': 21.85130615234375}

In [57]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.15826509960980395,
 'encoded_ood_sinkhorn_div_01': 21.725287119547527,
 'encoded_ood_e_distance': 2.5296382695145954,
 'encoded_ood_mmd': 10.320155461629232}

In [58]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7614227489499669,
 'decoded_ood_sinkhorn_div_01': 85.6328353881836,
 'decoded_ood_e_distance': 4.12626323148969,
 'decoded_ood_mmd': 21.937901814778645}

In [62]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results"

In [65]:
import os

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, "ood_metrics_encoded.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, "ood_metrics_decoded.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, "test_metrics_encoded.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, "test_metrics_decoded.csv"))
pd.DataFrame.from_dict(train_metrics_encoded).to_csv(os.path.join(output_dir, "train_metrics_encoded.csv"))
pd.DataFrame.from_dict(train_metrics_decoded).to_csv(os.path.join(output_dir, "train_metrics_decoded.csv"))

In [105]:
def rank_genes_groups_by_cov(
    adata,
    groupby,
    control_group,
    covariate,
    n_genes=50,
    rankby_abs=True,
    key_added="rank_genes_groups_cov",
    return_dict=False,
):
    gene_dict = {}
    cov_categories = adata.obs[covariate].unique()
    for cov_cat in cov_categories:
        # name of the control group in the groupby obs column
        control_group_cov = control_group  # "_".join([cov_cat, control_group])
        # subset adata to cells belonging to a covariate category
        adata_cov = adata[adata.obs[covariate] == cov_cat]
        # compute DEGs
        sc.tl.rank_genes_groups(
            adata_cov,
            groupby=groupby,
            reference=control_group_cov,
            rankby_abs=rankby_abs,
            n_genes=n_genes,
            use_raw=False,
        )
        # add entries to dictionary of gene sets
        de_genes = pd.DataFrame(adata_cov.uns["rank_genes_groups"]["names"])
        for group in de_genes:
            gene_dict[group] = de_genes[group].tolist()
    adata.uns[key_added] = gene_dict
    if return_dict:
        return gene_dict


def get_DE_genes(adata):
    adata.obs.loc[:, "control"] = adata.obs.condition.apply(lambda x: 1 if x == "control" else 0)
    adata.obs = adata.obs.astype("category")
    rank_genes_groups_by_cov(
        adata,
        groupby="condition",
        covariate="cell_type",
        control_group="control",
        n_genes=50,
        key_added="rank_genes_groups_cov_all",
    )
    return adata

In [106]:
adata.obs["cell_type"].value_counts()

cell_type
A549    63878
Name: count, dtype: int64

In [107]:
get_DE_genes(adata_train)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:398: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

AnnData object with n_obs × n_vars = 48279 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'ood', 'is_ood', 'cov_drug_dose', 'condition_ID', 'log_dose', 'split_1ct_MEC', 'CPA_cat', 'CPA_CHEMBL504', '_scvi_condition_ID', '_scvi_cell_type', '_scvi_CPA_cat', 'condition_name'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'rank_genes_groups_cov_all'
    obsm: 'CPA_pred', 'X_pca', 'X_umap', 'ecfp_drug_1', 'ecfp_drug_2', 'perts', 'perts_doses'
    varm: 'X_train_mean'
    layers: 'centered_X', 'counts'

In [111]:
mask = adata_train.var_names.isin(adata_train.uns["rank_genes_groups_cov_all"]["Alvespimycin+Pirarubicin"])

In [113]:
mask.shape

(2000,)

In [116]:
adata_train[adata_train.obs["condition"] == cond][:, mask].shape

(951, 50)

In [ ]:
predicted_target_ood_deg = jax.tree_util.tree_map(lambda x, y: x[:, y], predicted_target_ood_decoded, ood_masks)